In [1]:
!pip install selenium                 #install selenium 
!apt-get update -y                    # update 
!apt install chromium-chromedriver -y #install chrome driver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install beautifulsoup4           #install beautiful soup
import pandas as pd
import sys
import time
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


     |████████████████████████████████| 958 kB 4.2 MB/s 
     |████████████████████████████████| 356 kB 44.5 MB/s 
     |████████████████████████████████| 58 kB 3.7 MB/s 
Get:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5388 B]
Get:2 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [6786 B]
Get:3 http://packages.cloud.google.com/apt cloud-sdk InRelease [6745 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://packages.cloud.google.com/apt gcsfuse-bionic/main amd64 Packages [889 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://packages.cloud.google.com/apt cloud-sdk-bionic/main amd64 Packages [235 kB]
Get:9 http://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [216 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/univ

In [2]:
!mkdir test_images

In [3]:
import sys
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import urllib.request

# default path to file to store data
path_to_file = "reviews.csv"

total_photos = 0

# Import the webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

urls = ["https://www.tripadvisor.in/Hotel_Review-g186338-d192036-Reviews-The_Montague_on_The_Gardens-London_England.html",
       "https://www.tripadvisor.in/Hotel_Review-g186338-d188961-Reviews-Hotel_41-London_England.html",
        "https://www.tripadvisor.in/Hotel_Review-g186330-d279582-Reviews-Windermere_Boutique_Hotel-Windermere_Lake_District_Cumbria_England.html",
       "https://www.tripadvisor.in/Hotel_Review-g186338-d193121-Reviews-The_Milestone_Hotel_and_Residences-London_England.html",
       "https://www.tripadvisor.in/Hotel_Review-g186338-d195284-Reviews-Royal_Lancaster_London-London_England.html",
       "https://www.tripadvisor.in/Hotel_Review-g186525-d15380085-Reviews-Fingal-Edinburgh_Scotland.html",
       "https://www.tripadvisor.in/Hotel_Review-g186338-d2413342-Reviews-The_Hari-London_England.html",
       "https://www.tripadvisor.in/Hotel_Review-g186370-d193306-Reviews-The_Roseate_Villa_Bath-Bath_Somerset_England.html",
        "https://www.tripadvisor.in/Hotel_Review-g186566-d501257-Reviews-Craigatin_House_Courtyard-Pitlochry_Perth_and_Kinross_Scotland.html",
        "https://www.tripadvisor.in/Hotel_Review-g186370-d191735-Reviews-The_Kennard-Bath_Somerset_England.html"]

# Open the file to save the review
csvFile = open(path_to_file, 'w', encoding="utf-8")
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id', 'filename', 'img_url', 'traveller_name', 'date', 'hotel_name', 'review_title', 'review_text', 'rating'])
id = 0
start=time.time()
for url in urls:

  driver.get(url)
  hotel_name = driver.find_element(By.XPATH,'//*[@id="HEADING"]').text
  print(hotel_name)
  num_photos_text = driver.find_element(By.XPATH,'//*[@id="component_38"]/div/div[2]/div[1]/div/div/div/div/div/span[2]').text
  num_photos = int(num_photos_text[1:-1])
  total_photos = total_photos + num_photos
  print(num_photos)
  traveller_photos_tile = driver.find_element(By.XPATH,'//*[@id="component_38"]/div/div[2]/div[1]/div/div/div/div/div/span[1]')
  driver.execute_script("arguments[0].click();", traveller_photos_tile)
  time.sleep(2)
  first_photo = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/div/div[3]')
  driver.execute_script("arguments[0].click();", first_photo)  
  time.sleep(1)
  
  # expand the dialog for traveller photos 
  for j in range(2):        
      img_url = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[1]/div/div/div/div/picture/img').get_attribute("src")
      filename = str(id)+".jpg"
      urllib.request.urlretrieve(img_url, "test_images/" +filename)    
      traveller_name = ''
      traveller_date = ''
      traveller_contributions = ''
      try:
        name_and_date = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[3]/div[1]/div[1]/div[2]/span/a').text
        traveller_name = name_and_date.split(" • ")[0]
        traveller_date = name_and_date.split(" • ")[1]      
        traveller_contributions = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[3]/div[1]/div[1]/div[2]/div/div/div/span/span').text        
      except:
        name_and_date = ''
      review_text = ''
      review_title = ''      
      rating = ''
      try:
        review_dialog_tile = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[3]/div[2]/div[2]')
        driver.execute_script("arguments[0].click();", review_dialog_tile)     
        time.sleep(1)
        review_title = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[3]/div[2]/div[3]/div/div[2]/div[2]/span').text
        review_text = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[3]/div[2]/div[3]/div/div[2]/div[2]/p').text
        ratingBox = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div/div/div[3]/div[2]/div[3]/div/div[2]/div[1]')
        rating = ratingBox.find_element(By.XPATH,'.//*[local-name() = \'svg\']').get_attribute('aria-label')[:-13]
      except:
        review_text = ''
      csvWriter.writerow([id, filename, img_url, traveller_name, traveller_date, hotel_name, review_title, review_text, rating ]) 
      id = id + 1      
      try:
        next_button = driver.find_element(By.XPATH,'//*[@id="component_30"]/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[2]/button')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(0.1)
        if id % 10 == 0:
            print(id, time.time()-start)        
      except:
        continue

driver.close()

print("Total Photos: ", total_photos)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


The Montague on The Gardens
1681
Hotel 41
1761
Windermere Boutique Hotel
1269
The Milestone Hotel and Residences
1009
Royal Lancaster London
804
10 54.93287110328674
Fingal
852
The Hari
550
The Roseate Villa Bath
620
Craigatin House & Courtyard
381
The Kennard
320
20 108.17113590240479
Total Photos:  9247
